# Generador de personas aleatorias

In [169]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import unicodedata
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder

In [170]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
ciudades = pd.read_csv('./catalogos/ciudades.csv', encoding = 'UTF-16')

def quitar_acentos(s):
	return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [187]:
fake = Faker(['es_MX'])
personas = []
for _ in range(5):
	sexo = rm.choice(['H', 'M'])
	# generar tipo de sangre y asignar probabilidad a cada valor para que salga
	tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
	tipo_sangre = tipo_sangre[0]

	fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
	# calcular edad en base a la fecha de nacimiento
	dia_actual = dt.date.today()
	edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

	# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
	# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
	# mucha gente deja de crecer a los 20
	margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

	# dependiendo del sexo se generan nombres femeninos o masculinos
	# y estaturas y pesos dentro de su margen correspondiente
	if sexo == 'H':
		titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
		nombre = rm.choice(nombres['h'])
		altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
		peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
	else:
		titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
		nombre = rm.choice(nombres['m'])
		altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
		peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

	apellido_paterno = fake.last_name()
	apellido_materno = fake.last_name()

	# Generar un nombre de usuario para un email
	switcher = {
		1: fake.user_name(),
		# aperez
		2: f'{nombre[0]}{apellido_paterno}',
		# alonso_perez
		3: f'{nombre}_{apellido_paterno}',
		# aps70
		4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
		# aps1970
		5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
		# alonsoperezsoltero
		6: f'{nombre}{apellido_paterno}{apellido_materno}'
	}
	rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
	usuario = switcher.get(rand_int, 'usuario')
	# quitar espacios o reemplazarlos por un _
	usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
	usuario = quitar_acentos(usuario)

	subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
	correo = usuario + f'@example{subdominio}'

	id_estado = rm.choice(ciudades['id_estado'].unique())
	ct_ciudad = ciudades[ciudades['id_estado'] == id_estado].sample().values[0]

	ciudad = ct_ciudad[2]
	estado = ct_ciudad[1]
	tel = f'({ct_ciudad[3]})' + str(rm.randint(1000000, 9999999))

	direccion = fake.street_address()

	direcc_calle = fake.street_name()
	direcc_numero = fake.building_number()
	direcc_cp = fake.postcode()

	profesion = profesiones.sample().values[0][0] if edad >= 18 else None

	# TODO si el catalogo ya tiene coordenadas puestas, tomar esas

	# obtener latitud y longitud segun la ciudad
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	location = geolocator.geocode(f'{ciudad}, {estado}, Mexico')
	if location:
		lat = location.latitude
		lon = location.longitude
		# obtener zona horaria segun latitud y longitud
		obj = TimezoneFinder()
		timezone = obj.timezone_at(lng = location.longitude, lat = location.latitude)
	else:
		lat = lon = timezone = None

	personas.append({
		'titulo': titulo,
		'nombre': nombre,
		'apellido_paterno': apellido_paterno,
		'apellido_materno': apellido_materno,
		'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
		'sexo': sexo,
		'tipo_sangre': tipo_sangre,
		'altura': altura,
		'peso': peso,
		'correo': correo,
		'tel': tel,
		'calle': '',
		'numero': '',
		'cp': '',
		'ciudad': ciudad,
		'estado': estado,
		'lat': lat,
		'lon': lon,
		'zona_horaria': timezone,
		'profesion': profesion,
		'color_fav': fake.safe_color_name(),
	})
pd.DataFrame(personas)

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,tipo_sangre,altura,peso,correo,...,calle,numero,cp,ciudad,estado,lat,lon,zona_horaria,profesion,color_fav
0,Dra.,Itzel,Jaramillo,Montez,1992-09-19,M,A+,168.33,60.82,ijm92@example.org,...,,,,Felipe Neri (Cuatepec),Morelos,NaN,NaN,None,Bioquímico clínico,purple
1,None,Antonio,Saavedra,Rodríquez,1928-02-15,H,O+,181.54,58.93,antoniosaavedrarodriquez@example.net,...,,,,Tamazula de Victoria,Durango,24.969009,-106.967050,America/Monterrey,Trabajador de las artes comunitarias,gray
2,Sra.,Noelia,Grijalva,Nieto,1917-05-03,M,A+,172.87,81.08,ngn1917@example.gob,...,,,,Ixhuatlán del Café,Veracruz,19.026624,-96.927312,America/Mexico_City,Ajustador de siniestros colegiado,olive
3,Mtro.,Xavier,Lozada,Pelayo,1998-06-03,H,A+,166.86,76.48,xlozada@example.io,...,,,,Santiago,Baja California Sur,23.481372,-109.714600,America/Mazatlan,Barista,blue
4,None,Raquel,Maya,Guajardo,1928-03-19,M,AB-,165.43,77.67,raquel_maya@example.gob,...,,,,Coamiles,Nayarit,21.921329,-105.254371,America/Mazatlan,Periodista de radiodifusión,lime


In [184]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
	'results': 100,
	#'nat': 'es',
	#'password': '8-16,lower,upper,number,special',
	'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
	if isinstance(v, int): v = str(v)
	query += '&' + k + '=' + v
url = url_api + query

personas = pd.read_csv(url)
personas.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
56,male,Mr,Birk,Valderhaug,5747,Bryggegata,Arendal,Nordland,Norway,0440,55.1941,-101.4279,+6:00,"Almaty, Dhaka, Colombo",birk.valderhaug@example.com,1986-09-23T07:27:29.397Z,36,53371607,42579081
85,male,Mr,Daryl,Diaz,9276,Pockrus Page Rd,Albany,Queensland,Australia,8005,10.7341,100.3827,+5:45,Kathmandu,daryl.diaz@example.com,1970-04-08T19:19:38.244Z,52,06-6232-6252,0448-354-319
30,male,Monsieur,Mirco,Marchand,8927,Cours Charlemagne,Albinen,Graubünden,Switzerland,6604,-37.5475,6.5309,+1:00,"Brussels, Copenhagen, Madrid, Paris",mirco.marchand@example.com,1983-11-05T21:01:31.023Z,39,077 332 38 43,079 849 88 82
23,male,Mr,Timian,Horn,5011,Tåsenveien,Frogner,Aust-Agder,Norway,3725,63.0719,-59.6271,+8:00,"Beijing, Perth, Singapore, Hong Kong",timian.horn@example.com,1979-08-17T03:34:05.927Z,43,67940365,43000066
94,female,Madame,Celina,Lecomte,2729,Avenue de la République,Courroux,Obwalden,Switzerland,6506,-79.8239,70.1142,+6:00,"Almaty, Dhaka, Colombo",celina.lecomte@example.com,1961-08-29T10:30:07.010Z,61,079 154 62 70,075 509 81 45
92,female,Ms,Sophie,Cole,6304,Green Rd,Melbourne,Northern Territory,Australia,8498,-72.4738,18.9564,+9:00,"Tokyo, Seoul, Osaka, Sapporo, Yakutsk",sophie.cole@example.com,1994-05-23T12:33:48.602Z,28,02-4126-3787,0489-108-182
87,female,Miss,Olivia,Brown,5999,Stuart Street,Rotorua,Wellington,New Zealand,76858,81.3708,-84.4374,+11:00,"Magadan, Solomon Islands, New Caledonia",olivia.brown@example.com,1963-01-29T02:56:26.430Z,59,(137)-771-2991,(110)-352-6079
61,male,Mr,Léonard,Jean,3472,Boulevard de la Duchère,Lyon,Loiret,France,75250,33.8556,-12.3204,0:00,"Western Europe Time, London, Lisbon, Casablanca",leonard.jean@example.com,1978-06-28T13:59:42.263Z,44,02-35-50-81-81,06-37-84-12-75
